In [1]:
import ultralytics
ultralytics.checks()
import json
import cv2
import os
from sklearn.model_selection import train_test_split
import shutil
from ultralytics import YOLO
from pprint import pprint
from pathlib import Path
import random
from collections import defaultdict

Ultralytics 8.3.146  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (GeForce GTX 1650, 4096MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 140.7/931.5 GB disk)


Only run this notebook once, after downloading the dataset

In [ ]:
# how to get COCO formatted data: 
# download the dataset from https://datasetninja.com/mju-waste#download and place in folder 'data/mju-waste-COCO
# clone the instances.json files from https://github.com/realwecan/mju-waste and place in folder 'data/mju-waste-COCO/annotations'

In [2]:

def show_first_two_per_category(json_path):
    """
    Prints the first two entries of each root-level list in a JSON file.

    Useful for quickly inspecting the structure and content of a COCO-style
    annotations json file.

    It pretty-prints the first two entries of each top-level key that contains a list.

    Args:
        json_path (str or Path): Path to the JSON file to inspect.

    Raises:
        FileNotFoundError: If the provided path does not point to an existing file.
        json.JSONDecodeError: If the file is not valid JSON.
    """
    json_path = Path(json_path)

    if not json_path.exists():
        print(f"File not found: {json_path}")
        return

    with open(json_path, 'r') as f:
        data = json.load(f)

    for key, value in data.items():
        print(f"\n--- {key.upper()} (showing first 2 entries) ---")
        if isinstance(value, list):
            for item in value[:2]:
                pprint(item)
        else:
            print(f"{key} is not a list, skipping.")


show_first_two_per_category('..\\data\\mju-waste-COCO\\annotations\\train.json')


--- INFO (showing first 2 entries) ---
info is not a list, skipping.

--- LICENSES (showing first 2 entries) ---
licenses is not a list, skipping.

--- CATEGORIES (showing first 2 entries) ---
{'id': 0, 'name': 'Rubbish', 'supercategory': 'Waste'}

--- ANNOTATIONS (showing first 2 entries) ---
{'area': 11013.695949999998,
 'bbox': [450, 255, 137, 204],
 'category_id': 0,
 'id': 1621,
 'image_id': 1617,
 'iscrowd': 0,
 'segmentation': [318.11,
                  188.75,
                  332.25,
                  324.48,
                  414.96,
                  315.99,
                  395.17,
                  180.27,
                  318.11,
                  188.04]}
{'area': 13896.986949999991,
 'bbox': [403, 228, 149, 198],
 'category_id': 0,
 'id': 1622,
 'image_id': 1618,
 'iscrowd': 0,
 'segmentation': [287.01,
                  164.71,
                  385.98,
                  162.59,
                  390.22,
                  296.2,
                  285.6,
           

In [3]:
# This script combines the train, val, and test JSON files from the MJU Waste dataset into a single COCO-style JSON file.
# it also cleans the annotation files, only keeping annotation id, image_id, category_id (set to 0 for 'trash'), bbox, area, and iscrowd.

# Set paths
input_dir = Path("..") / "data" / "mju-waste-COCO" / "annotations"
output_file = Path("..") / "data" / "mju-waste-COCO" / "clean_annotations" / "annotations.json"
output_file.parent.mkdir(parents=True, exist_ok=True)

# Files to process
splits = ['train', 'val', 'test']

combined_images = []
combined_annotations = []
annotation_id = 0
image_ids_seen = set()

for split in splits:
    input_file = input_dir / f"{split}.json"

    with input_file.open('r') as f:
        data = json.load(f)

    for img in data.get("images", []):
        if img["id"] not in image_ids_seen:
            combined_images.append(img)
            image_ids_seen.add(img["id"])

    for ann in data.get("annotations", []):
        cleaned_ann = {
            'id': annotation_id,
            'image_id': ann['image_id'],
            'category_id': 0,  # unify to 'trash'
            'bbox': ann['bbox'],
            'area': ann['area'],
            'iscrowd': ann.get('iscrowd', 0)
        }
        combined_annotations.append(cleaned_ann)
        annotation_id += 1

# Set single category at the end
categories = [{"id": 0, "name": "trash"}]

# Build and save final dataset
cleaned_data = {
    'images': combined_images,
    'annotations': combined_annotations,
    'categories': categories
}

with output_file.open('w') as f:
    json.dump(cleaned_data, f)

print(f"Saved combined cleaned annotations to {output_file}")
print(f"Total images: {len(combined_images)}")
print(f"Total annotations: {len(combined_annotations)}")

Saved combined cleaned annotations to ..\data\mju-waste-COCO\clean_annotations\annotations.json
Total images: 2475
Total annotations: 2532


In [4]:
show_first_two_per_category('..\\data\\mju-waste-COCO\\clean_annotations\\annotations.json')



--- IMAGES (showing first 2 entries) ---
{'coco_url': '',
 'date_captured': '2019-11-21 16:19:37',
 'file_name': '2019-09-19_16_19_32-29_color.png',
 'flickr_url': '',
 'height': 480,
 'id': 1617,
 'license': 1,
 'width': 640}
{'coco_url': '',
 'date_captured': '2019-11-21 16:19:37',
 'file_name': '2019-09-19_16_19_44-93_color.png',
 'flickr_url': '',
 'height': 480,
 'id': 1618,
 'license': 1,
 'width': 640}

--- ANNOTATIONS (showing first 2 entries) ---
{'area': 11013.695949999998,
 'bbox': [450, 255, 137, 204],
 'category_id': 0,
 'id': 0,
 'image_id': 1617,
 'iscrowd': 0}
{'area': 13896.986949999991,
 'bbox': [403, 228, 149, 198],
 'category_id': 0,
 'id': 1,
 'image_id': 1618,
 'iscrowd': 0}

--- CATEGORIES (showing first 2 entries) ---
{'id': 0, 'name': 'trash'}


In [ ]:
# This cell sets up the function that converts the simple COCO dataset to YOLO format

def convert_coco_to_yolo(coco_root: Path, dataset_name: str, train_split: float = 0.8):
    """
    Converts a simple COCO dataset to YOLOv8 format, including train/val split and data.yaml generation.

    Args:
        coco_root (Path): Path to the root of the simple COCO dataset (should contain images/ and annotations.json).
        dataset_name (str): Name of the output dataset folder (e.g., "taco" -> creates "taco_yolo").
        train_split (float, optional): Fraction of images to use for training. Defaults to 0.8.
          The remaining images are split between validation and testing.

    Returns:
        Path: Path to the data.yaml file
    """
    # Paths
    coco_json_path = coco_root / 'annotations.json'
    coco_images_path = coco_root / 'images'

    # Load COCO JSON and get number of images for naming
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)
    n_total = len(coco['images'])

    # Paths con't
    yolo_root = coco_root.parent / f"{dataset_name}_yolo_{n_total}"
    yolo_img_dirs = {
        'train': yolo_root / 'images' / 'train',
        'val': yolo_root / 'images' / 'val',
        'test': yolo_root / 'images' / 'test',
    }
    yolo_lbl_dirs = {
        'train': yolo_root / 'labels' / 'train',
        'val': yolo_root / 'labels' / 'val',
        'test': yolo_root / 'labels' / 'test',
    }

    # Clear and recreate folders
    for d in list(yolo_img_dirs.values()) + list(yolo_lbl_dirs.values()):
        if d.exists():
            shutil.rmtree(d)
        d.mkdir(parents=True, exist_ok=True)

    # Map image_id -> metadata
    image_info = {img['id']: (img['width'], img['height'], img['file_name']) for img in coco['images']}

    # Map image_id -> annotations
    annots_per_image = defaultdict(list)
    for ann in coco['annotations']:
        annots_per_image[ann['image_id']].append(ann)

    # Shuffle and split image IDs
    all_image_ids = list(image_info.keys())
    random.shuffle(all_image_ids)
    n_total = len(all_image_ids)
    n_train = int(n_total * train_split)
    n_val = int((n_total - n_train) / 2)
    n_test = n_total - n_train - n_val

    split_ids = {
        'train': set(all_image_ids[:n_train]),
        'val': set(all_image_ids[n_train:n_train + n_val]),
        'test': set(all_image_ids[n_train + n_val:]),
    }

    def write_labels_and_copy_images(image_ids, img_dir, lbl_dir):
        for image_id in image_ids:
            width, height, filename = image_info[image_id]
            orig_stem = Path(filename).stem
            new_stem = f"{dataset_name}_{orig_stem}"
            label_path = lbl_dir / f"{new_stem}.txt"
            image_src = coco_images_path / filename
            image_dst = img_dir / f"{new_stem}.jpg"

            # Copy image
            if image_src.exists():
                shutil.copy(image_src, image_dst)
            else:
                print(f"Warning: Image not found: {image_src}")
                continue

            # Write labels
            with open(label_path, 'w') as f:
                for ann in annots_per_image.get(image_id, []):
                    class_id = ann['category_id']
                    x, y, w, h = ann['bbox']
                    x_center = (x + w / 2) / width
                    y_center = (y + h / 2) / height
                    w /= width
                    h /= height
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    # Process splits
    for split in ['train', 'val', 'test']:
        write_labels_and_copy_images(
            split_ids[split],
            yolo_img_dirs[split],
            yolo_lbl_dirs[split]
        )


    print(f"YOLO conversion complete: {yolo_root}")
    print(f"  Train: {len(split_ids['train'])}, Val: {len(split_ids['val'])}, Test: {len(split_ids['test'])}")

In [24]:
convert_coco_to_yolo(
    coco_root= Path("..\\data\\mju-COCO"),
    dataset_name="mju_waste",
    train_split=0.8
)

YOLO conversion complete: ..\data\mju_waste_yolo_2475
  Train: 1980, Val: 247, Test: 248


In [25]:
images_dir

WindowsPath('../data/mju_waste_yolo_2475/images/train')

In [26]:

dataset_dir = Path("..\\data\mju_waste_yolo_2475")
images_dir = dataset_dir / "images" / "train"
labels_dir = dataset_dir / "labels" / "train"
output_dir = Path("..\\results")
class_names = ["trash"]

# Get all image paths and sample 3
image_paths = list(images_dir.glob("*.jpg"))
random_images = random.sample(image_paths, min(3, len(image_paths)))

for image_path in random_images:
    label_path = labels_dir / (image_path.stem + ".txt")

    # Read image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not load image: {image_path}")
        continue
    h, w = image.shape[:2]

    # Read and draw bounding boxes
    if label_path.exists():
        with label_path.open("r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                cls, x_center, y_center, box_w, box_h = map(float, parts)
                x_center *= w
                y_center *= h
                box_w *= w
                box_h *= h

                x1 = int(x_center - box_w / 2)
                y1 = int(y_center - box_h / 2)
                x2 = int(x_center + box_w / 2)
                y2 = int(y_center + box_h / 2)

                # Draw bounding box and class label
                label = class_names[int(cls)] if int(cls) < len(class_names) else str(int(cls))
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)
    else:
        print(f"No label found for {image_path.name}")

    # Show image
    output_path = output_dir / image_path.name
    cv2.imwrite(str(output_path), image)
    print(f"✅ Saved annotated image to: {output_path}")

cv2.destroyAllWindows()

✅ Saved annotated image to: ..\results\mju_waste_2019-09-24_16_30_00-69_color.jpg
✅ Saved annotated image to: ..\results\mju_waste_2019-10-15_18_22_36-53_color.jpg
✅ Saved annotated image to: ..\results\mju_waste_2020-01-07_17_31_45-52_color.jpg
